<div class="jumbotron">
  <h1><i class="fa fa-bar-chart" aria-hidden="true"></i>Clase prototipo</h1>
  <p></p>
</div>

El prototipo es una clase en código python con el modelo seleccionado del notebook 5.
El cual incluye los métodos para 

- Lectura de los datos de entrenamiento (csv)
- Validar datos y definir variables de clase (target, nombre de features, ...)
- Preprocesar los datos de entrenamiento(segun el notebook 4)
- Almacenar modelos de pre procesamiento (z-score , label coding , one hot coding)
- Leer los modelos de pre procesamientO 
- Preprocesar datos para nuevos casos (productivo)
- Entrar modelo modelo(segun el notebook 5)
- Guardar modelo en formato pickle
- Cargar el modelo entrenado
- run : funciones pricipal el cual es el pipeline del proceso completo(load, pre procesado, entraniento,...)
- Predicir nuevos casos 


In [2]:
from os import chdir
chdir("..")

In [3]:
ls

api.py         categorizador/ data/          experimentos/  model/


In [3]:
! pip freeze > requirements.txt

In [87]:
# %%writefile categorizador/classifier.py


from pandas import read_csv,concat
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump, load
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas import DataFrame
from numpy import where, number
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from joblib import dump, load

# Función para el tratamiento de outliers por cuartiles y desviación estándar
def treat_outliers(data, numeric_columns, factor=1.5):
    treated_data = data.copy()
    for column in numeric_columns:
        q1 = data[column].quantile(0.25)
        q3 = data[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - factor * iqr
        upper_bound = q3 + factor * iqr
        treated_data[column] = where(treated_data[column] < lower_bound, lower_bound, treated_data[column])
        treated_data[column] = where(treated_data[column] > upper_bound, upper_bound, treated_data[column])
    return treated_data

class Classifier:
    """

    
    Clase para procesar datos, entrenar un modelo de RandomForest y realizar predicciones.

    Métodos
    -------
    preprocess_data():
        Método para preprocesar datos. (Personalizar según necesidad)

    transform_data():
        Método para transformar los datos (imputación, escalado, codificación, etc.)

    split_data():
        Divide los datos en conjuntos de entrenamiento y prueba.

    train_model(X_train, y_train):
        Entrena el modelo RandomForest con los datos de entrenamiento.

    evaluate_model(X_test, y_test):
        Evalúa el modelo en los datos de prueba y devuelve la precisión.

    save_model(filename):
        Guarda el modelo entrenado y los objetos de preprocesamiento en archivos.

    load_model(filename):
        Carga el modelo y los objetos de preprocesamiento desde archivos.

    retrain_model(new_data_path):
        Reentrena el modelo con nuevos datos.

    treat_outliers(data, numeric_columns, factor=1.5):
        Trata los outliers en las columnas numéricas.

    transform_for_prediction(X):
        Transforma los datos nuevos para predicción utilizando los mismos pasos de preprocesamiento.

    predict(new_data):
        Realiza predicciones sobre nuevos datos.
    """
    def __init__(self):
        
        """
        Inicializa la clase Classifier.

        Parameters
        ----------
        data_path : str, optional
            Ruta al archivo CSV que contiene los datos. Si no se proporciona, se debe cargar manualmente.
        """
        
        self.model = None
        # Convertir las columnas a su dtype correcto 
        self.numeric_columns = ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
        self.categorical_columns = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
        self.label = ['Survived']

        self.columns = ['PassengerId', 
                        'Pclass',
                        'Name', 
                        'Sex',
                        'Age',
                        'SibSp', 
                        'Parch',
                        'Ticket', 
                        'Fare',
                        'Cabin', 
                        'Embarked']

        self.class_dict = {0: 'No sobrebvivio', 1: 'Sobrevivio'}
        
    def loader(self, data_path):

        self.data = read_csv(data_path)
        self.numeric_imputer = SimpleImputer(strategy='mean')
        self.categorical_imputer = SimpleImputer(strategy='most_frequent')
        self.scaler = StandardScaler()
        self.encoder = OneHotEncoder(handle_unknown='ignore')
        self.scaler = StandardScaler()
        self.encoder = OneHotEncoder(handle_unknown='ignore')
    
    def preprocess_data(self):
        self.data = treat_outliers(self.data, self.numeric_columns)
    
    def transform_data(self):

        # Imputar datos faltantes
        self.data[self.numeric_columns] = self.numeric_imputer.fit_transform(self.data[self.numeric_columns])
        self.data[self.categorical_columns] = self.categorical_imputer.fit_transform(self.data[self.categorical_columns])
        
        # Escalar características numéricas
        self.data[self.numeric_columns] = self.scaler.fit_transform(self.data[self.numeric_columns])
        
        # Codificar características categóricas
        encoded_categories = self.encoder.fit_transform(self.data[self.categorical_columns]).toarray()
        encoded_df = DataFrame(encoded_categories, columns=self.encoder.get_feature_names_out(self.categorical_columns))
        
        
        numeric_columns_extend = self.numeric_columns + ['Survived']
        # Concatenar datos transformados
        self.data = concat([self.data[numeric_columns_extend], encoded_df], axis=1)

    
    def split_data(self):
        X = self.data.drop(columns=['Survived'])
        y = self.data['Survived']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        return X_train, X_test, y_train, y_test
    
    def train_model(self, X_train, y_train):
        self.model = RandomForestClassifier()
        self.model.fit(X_train, y_train)
        
    
    def evaluate_model(self, X_test, y_test):
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy
    
    def save_model(self, filename):
        
        filename = 'model/' + filename
        
        dump(self.model, filename)
        
        dump(self.numeric_imputer, filename + '_numeric_imputer.joblib')
        dump(self.categorical_imputer, filename + '_categorical_imputer.joblib')
        dump(self.scaler, filename + '_scaler.joblib')
        dump(self.encoder, filename + '_encoder.joblib')
    
    def load_model(self, filename):
        self.model = load(filename)
        self.numeric_imputer = load(filename + '_numeric_imputer.joblib')
        self.categorical_imputer = load(filename + '_categorical_imputer.joblib')
        self.scaler = load(filename + '_scaler.joblib')
        self.encoder = load(filename + '_encoder.joblib')
    
    def transform_for_prediction(self, X):
        # Tratamiento de outliers
        X = treat_outliers(X, self.numeric_columns)
        
        # Imputar valores faltantes
        X[self.numeric_columns] = self.numeric_imputer.transform(X[self.numeric_columns])
        X[self.categorical_columns] = self.categorical_imputer.transform(X[self.categorical_columns])

        # Escalar características numéricas
        X[self.numeric_columns] = self.scaler.transform(X[self.numeric_columns])

        # One-Hot Encoding para características categóricas
        encoded_cats = self.encoder.transform(X[self.categorical_columns]).toarray()
        encoded_cat_df = DataFrame(encoded_cats, columns=self.encoder.get_feature_names_out(self.categorical_columns))
        
        X = X.drop(columns=self.categorical_columns).reset_index(drop=True)
        X = concat([X, encoded_cat_df], axis=1)

        return X
    
    def predict(self, new_data):

        
        # Asegurarse de que los nuevos datos sean un DataFrame de pandas
        if not isinstance(new_data, DataFrame):
            new_data = DataFrame(new_data, columns= self.columns)
        
        # Transformar los datos nuevos para predicción
        X_new = self.transform_for_prediction(new_data)
        
        # Hacer predicciones
        predictions = self.model.predict(X_new)

        """
        predictions_proba = self.model.predict_proba(X_new)
        predicts_ = []
        for prediction in predictions:
            predicts_.append(self.class_dict[prediction])

        """
            
        
        return predictions

### Ejemplo de Uso
- Entrenamiento y guardado del modelo:
  ```python
  data_processor = DataProcessor()
  data_processor.loader('titanic.csv')
  data_processor.preprocess_data()
  data_processor.transform_data()
  X_train, X_test, y_train, y_test = data_processor.split_data()
  data_processor.train_model(X_train, y_train)
  data_processor.save_model('trained_model.joblib')
  ```

- Cargar el modelo y hacer predicciones con nuevos datos:
  ```python
  data_processor.load_model('trained_model.joblib')
  new_data = pd.read_csv('new_titanic_data.csv')
  predictions = data_processor.predict(new_data)
  print(predictions)
  ```

In [28]:
! ls

api.py        categorizador data          experimentos  model


In [88]:
path= 'data/raw/titanic.csv'

# Ejemplo de uso
data_processor = Classifier()
data_processor.loader(path)
data_processor.preprocess_data()
data_processor.transform_data()
X_train, X_test, y_train, y_test = data_processor.split_data()
data_processor.train_model(X_train, y_train)
# Suponiendo que tienes nuevos datos en un archivo llamado 'nuevos_datos.csv'
accuracy = data_processor.evaluate_model(X_test, y_test)
print("Accuracy:", accuracy)
salida  = data_processor.save_model('titanic_model')


Accuracy: 0.8100558659217877


In [9]:
!ls

api.py        categorizador data          experimentos  model


In [31]:
path= 'data/raw/titanic.csv'

data_processor = Classifier()
data_processor.load_model('model/titanic_model')
new_data = read_csv(path)
del new_data['Survived']
data_processor.predict(new_data)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

### Datos nuevos

In [32]:
nuevo_dato = new_data.iloc[0:10]

In [33]:
nuevo_dato

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [34]:
data_processor = Classifier()
data_processor.load_model('model/titanic_model')

predictions = data_processor.predict(nuevo_dato)

In [35]:
predictions

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1])

### Como numpy array 

In [41]:
data_array  = nuevo_dato.values

In [42]:
type(data_array)

numpy.ndarray

In [44]:
data_processor = Classifier()
data_processor.load_model('model/titanic_model')
data_processor.predict(data_array)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1])

## ¡¡¡ cuando es numpy.ndarray te cuidado con el orden de los datos !!!!

## API 

In [86]:
%%writefile api.py
from flask import Flask, jsonify, request
from jsonschema import validate
from pandas import DataFrame
# Importa la clase Classifier de tu módulo
from categorizador.classifier import Classifier

app = Flask(__name__)

# Cargar la instancia de Classifier con el modelo entrenado
data_processor = Classifier()
data_processor.load_model('model/titanic_model')

# Esquema JSON para validar las entradas
input_schema = {
    "type": "object",
    "properties": {
        "PassengerId": {"type": "number"},
        "Pclass": {"type": "number"},
        "Name": {"type": "string"},
        "Sex": {"type": "string"},
        "Age": {"type": "number"},
        "SibSp": {"type": "number"},
        "Parch": {"type": "number"},
        "Ticket": {"type": "string"},
        "Fare": {"type": "number"},
        "Cabin": {"type": "string"},
        "Embarked": {"type": "string"}
    },
    "required": ["PassengerId", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
}

# Ruta para realizar predicciones
@app.route('/predict', methods=['POST'])
def predict():
    # Obtener los datos de form-data de la solicitud
    features = {
        "PassengerId": int(request.form['PassengerId']),
        "Pclass": int(request.form['Pclass']),
        "Name": request.form['Name'],
        "Sex": request.form['Sex'],
        "Age": float(request.form['Age']),
        "SibSp": int(request.form['SibSp']),
        "Parch": int(request.form['Parch']),
        "Ticket": request.form['Ticket'],
        "Fare": float(request.form['Fare']),
        "Cabin": request.form['Cabin'],
        "Embarked": request.form['Embarked']
    }

    # Validar las entradas con el esquema definido
    try:
        validate(instance=features, schema=input_schema)
    except Exception as e:
        return jsonify({'error': str(e)}), 400

    # Crear un DataFrame a partir de los datos recibidos
    df = DataFrame([features])

    # Realizar la predicción con el modelo cargado
    predictions = data_processor.predict(df)
    
    print(predictions)
    # Devolver la respuesta en formato JSON
    return jsonify({"response": predictions.tolist()})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)


Overwriting api.py


### Test API


```bash
curl -X POST -F "PassengerId=123" \
-F "Pclass=1" \
-F "Name=John Doe" \
-F "Sex=male" \
-F "Age=30" \
-F "SibSp=0" \
-F "Parch=0" \
-F "Ticket=12345" \
-F "Fare=50.5" \
-F "Cabin=C123" \
-F "Embarked=S" \
http://localhost:8080/predict
```

